In [17]:
from copy import copy, deepcopy
from pprint import pprint
from typing import List

def as_position_matrix(state, include_None=False):
    return sorted([
        (item, (i, j)) for i, row in enumerate(state)
        for j, item in enumerate(row) if (include_None or item is not None)
    ])

initial_state = [
    [1, 2, 3],
    [None, 4, 6],
    [7, 5, 8],
]

objective_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, None],
]

objective_state_position_matrix = as_position_matrix(objective_state)

class Puzzle8Solver():


    def solve(self):
        print(f'Initial state \n{initial_state}\n')
        print(f'Objective state  \n{objective_state}\n')

        initial_node = Node(initial_state)

        decision_border = DecisionBorder([
            Path([initial_node])
        ])
        visited_nodes = NodesList([])

        steps = 0
        number_of_nodes_created = 0
        decision_border_max_size = len(decision_border)

        while True:
            steps += 1

            decision_border_size = len(decision_border)
            if decision_border_size > decision_border_max_size:
                decision_border_max_size = decision_border_size

            # Get cheapest path
            cheapest_path = decision_border.cheapest_path()

            decision_node = cheapest_path.decision_node

            if visited_nodes.contains(decision_node):
                decision_border.remove_cheapest_path()
                continue

            visited_nodes.append(decision_node)
            if decision_node.is_objective():
                break

            # Generate children
            children = decision_node.generate_children()
            number_of_nodes_created += len(children)

            # Remove cheapest path
            decision_border.remove_cheapest_path()

            # Add new paths to decision border
            for child in children:
                new_path = copy(cheapest_path)
                new_path.append(child)
                decision_border = add_path(decision_border, new_path)

            #print('==================================')
            #print(f'decision_border \n{decision_border}\n')
            #print(f'Cheapest Path \n{cheapest_path}\n')
            #print(f'decision_node \n{decision_node}\n')
            #print(f'decision_node heuristic \n{decision_node.heuristic()}\n')
            #print(f'visited_nodes \n{visited_nodes}\n')
            #print(f'children \n{children}\n')
            #print('\n')

        print('\n\n\n\n')
        print('Found solution!\n')

        print(f'Iterations: {steps}\n')
        print(f'Number of visited nodes: {len(visited_nodes)}\n')
        print(f'Number of created nodes: {number_of_nodes_created}\n')
        print(f'Maximum decision border size: {decision_border_max_size}\n')
        print(f'Cheapest path size: {len(cheapest_path)}\n')

        visited_nodes_formated = '\n\n'.join([str(node) for node in visited_nodes])
        print(f'\nVisited nodes: \n{visited_nodes_formated}\n')
        print('\n------------------\n\n')
        print(f'Cheapest path:\n{cheapest_path}')
        

class Node():
    def __init__(self, state):
        self.state = state
        
    def __repr__(self):
        return '\n'.join([str(row) for row in self.state])
    
    @property
    def flat_state(self):
        return [item for sublist in self.state for item in sublist]
    
    def boolean_heuristic(self):
        """Sums number of pieces not in their objective position."""
        return sum([
            item != objective_state[i][j] for i, row in enumerate(self.state) for j, item in enumerate(row)
        ])
    
    def manhattan_distance(self, position_matrix_1, position_matrix_2):
        return sum([
            abs(i1 - i2) + abs(j1 - j2)
            for (_, (i1, j1)), (_, (i2, j2)) in zip(
                sorted(position_matrix_1), sorted(position_matrix_2)
            )
        ])
    
    def manhattan_heuristic(self):
        """Sums manhattan distances of all items to their objective."""
        return self.manhattan_distance(
            as_position_matrix(self.state),
            objective_state_position_matrix,
        )
    
    def heuristic(self):
        return self.manhattan_heuristic()
    
    def is_objective(self):
        return self.state == objective_state
        
    def empty_space_position(self):
        return next(
            ((i, j) for i, row in enumerate(self.state) for j, item in enumerate(row) if item is None)
        )
    
    def neighbors_positions(self, i, j):
        i_up = (max(0, i-1), j)
        i_down = (min(len(self.state)-1, i+1), j)
        j_left = (i, max(0, j-1))
        j_right = (i, min(len(self.state[0])-1, j+1))
        
        return [pos for pos in (i_up, i_down, j_left, j_right) if pos != (i, j)]
        
    def generate_children(self):
        i, j = self.empty_space_position()
        children = []
        for n_i, n_j in self.neighbors_positions(i, j):
            child_state = deepcopy(self.state)
            child_state[i][j] = self.state[n_i][n_j]
            child_state[n_i][n_j] = None
            children.append(
                type(self)(child_state)
            )
        return children

class NodesList(list):
    def contains(self, node_to_find):
        return any([node_to_find.state == node.state for node in self])
    
class Path(NodesList):
    def __repr__(self):
        return '\n    |\n    V\n'.join([str(node) for node in self])

    @property
    def decision_node(self):
        return self[-1]
    
    def cost(self):
        return len(self) + self.decision_node.heuristic()
    
class DecisionBorder(List[Path]):
    def __repr__(self):
        a = '\n============\n============\n'.join([str(path) for path in self])
        return f"[\n{a}\n]"
    
    def cheapest_path(self):
        return self[0]
    
    def remove_cheapest_path(self):
        self.pop(0)
        
def add_path(decision_border: DecisionBorder, new_path: Path) -> DecisionBorder:
    """Inserts Path into DecisionBorder, making DecisionBorder a list of ordered Path's by cost."""

    if len(decision_border) == 0:
        return DecisionBorder([new_path])

    for path_index, path in enumerate(decision_border):
        if path.cost() > new_path.cost():
            break

    return DecisionBorder(decision_border[:path_index] + [new_path] + decision_border[path_index:])

In [18]:
Puzzle8Solver().solve()

Initial state 
[[1, 2, 3], [None, 4, 6], [7, 5, 8]]

Objective state  
[[1, 2, 3], [4, 5, 6], [7, 8, None]]






Found solution!

Iterations: 4

Number of visited nodes: 4

Number of created nodes: 10

Maximum decision border size: 8

Cheapest path size: 4


Visited nodes: 
[1, 2, 3]
[None, 4, 6]
[7, 5, 8]

[1, 2, 3]
[4, None, 6]
[7, 5, 8]

[1, 2, 3]
[4, 5, 6]
[7, None, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, None]


------------------


Cheapest path:
[1, 2, 3]
[None, 4, 6]
[7, 5, 8]
    |
    V
[1, 2, 3]
[4, None, 6]
[7, 5, 8]
    |
    V
[1, 2, 3]
[4, 5, 6]
[7, None, 8]
    |
    V
[1, 2, 3]
[4, 5, 6]
[7, 8, None]
